In [2]:
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import seaborn as sea
import datetime
%matplotlib inline 

# Загрузка всех таблиц из файла data

In [3]:
data_work_time = pd.read_excel('data.xlsx', sheet_name='Рабочее время')
data_employee = pd.read_excel('data.xlsx', sheet_name='Сотрудник Подр. Месяц')
data_service = pd.read_excel('data.xlsx', sheet_name='Сервисы использование')
data_hierarchy = pd.read_excel('data.xlsx', sheet_name='Иерархия подразделений')

# Переименнуем именна столбцов всех таблиц
### Преобразуем данные для комфортной дальнейшей работы

In [4]:
#переименнуем именна столбцов всех таблиц на англиский язык для дальнейшего удобства
data_work_time = data_work_time.rename(columns = {'Дата' : 'date',
                                'Сотрудник' : 'employee',
                                'Рабочие часы' : 'working_hours',
                                'Подразделение' : 'subdivision_work',
                                'Подразделение_TXT' : 'subdivision_txt'})

data_employee = data_employee.rename(columns = {'Сотрудник' : 'employee',
                                'Подразделение' : 'subdivision_salary',
                                'Подразделение_TXT' : 'subdivision_txt',
                                'Месяц' : 'month',
                                'Должность' : 'position',
                                'Должность_TXT' : 'position_txt',
                                'Пол' : 'gender',
                                'ФИО' : 'full_name',
                                'ЗП (руб.)' : 'salary'})

data_service = data_service.rename(columns = {'Подразделение' : 'subdivision_work',
                               'Подразделение_TXT' : 'subdivision_txt',
                               'Должность' : 'position',
                               'Должность_TXT' : 'position_txt',
                               'Сервис' : 'service',
                               'Сервис_TXT' : 'service_txt',
                               '% использования в рабочее время' : 'persent_use_in_work_time'})

data_hierarchy = data_hierarchy.rename(columns = {'Ид. узла' : 'id_node',
                                 'Ид. подразделения' : 'id_subdivision',
                                 'Наименование' : 'subdivision_txt',
                                 'Родительский узел' : "parent's_node",
                                 'Следущий узел' : 'next_node'})

#добавим в первую таблицу месяца работы
data_work_time['month'] = [data_work_time['date'].iloc[i].month for i in range(data_work_time.shape[0])]

#преобразуем данные второй таблицы чтобы вместо полной даты остался только месяц
data_employee['month'] = [data_employee['month'].iloc[i].month for i in range(data_employee.shape[0])]

#удалим из таблиц данные о названиях должности, сервисов, подразделения, но до этого сохраним их в виде словаря
# id_txt : txt

sub = {}
for i in range(len(list(data_work_time['subdivision_work'].unique())) ):
    sub[list(data_work_time['subdivision_work'].unique()) [i]] = list(data_work_time['subdivision_txt'].unique())[i]

data_position = {}
for i in range(len(list(data_employee['position'].unique())) ):
    data_position[list(data_employee['position'].unique())[i]] = list(data_employee['position_txt'].unique())[i]
    
data_services = {}
for i in range(len(list(data_service['service'].unique()))):
    data_services[list(data_service['service'].unique())[i]] = list(data_service['service_txt'].unique())[i]

#так как в четвертой таблице данные о названиях повторяются, то возьмем два столбца о id и соответствуюющем названии
data_subdivision = data_hierarchy[['id_subdivision', 'subdivision_txt']]

#теперь когда данные сохраненны в отдельные переменные можем удалить лишнии данные из таблиц
data_work_time = data_work_time.drop(['subdivision_txt'], axis = 1)

data_employee = data_employee.drop(['subdivision_txt', 'position_txt', 'full_name'], axis = 1)

data_service = data_service.drop(['subdivision_txt', 'position_txt', 'service_txt'], axis = 1)

data_hierarchy = data_hierarchy.drop(['subdivision_txt'], axis = 1)

#теперь данные преобразованны и можно переходить к их обработке

# Рассмотрим статистические данные, которые можем получить на основе предоставленных данных

### Найдем среднюю зарплату для каждого подразделения в зависимости от должности и пола

In [5]:
date_mean_salary = data_employee.groupby(['subdivision_salary', 'position', 'gender']).agg({'salary' : 'mean'})

### Соединим первую и вторую таблицу по id_работника и месяцу работы

In [6]:
data_work_time = data_work_time.merge(data_employee, on =('employee', 'month'))

### Составим таблицу в которой показанно рабочие часы каждого работника в соответствующем месяце

In [7]:
time_work_of_employee = data_work_time.groupby(['employee', 'month']).agg({'working_hours' : 'sum'})

### Присоединим третью таблицу и сгруппируем по подразделению и по должности
- ###### Получим таблицу с данными о кол-ве рабочих часов в месяц и суммарную зарплату и столбик со значениями зарплаты полученной в час на данном сервисе

Создадим таблицу с данными, где: 
- working_hours - это суммарное время работы всех сотрудников в соответствующем месяце, на соответ. должности, в соотв. подразделении
- salary - суммарная зарплата всех сотрудников в соответствующем месяце, на соответ. должности, в соотв. подразделении
- money_in_hour - деньги получаемые в час на соответствующем сервисе для всех сотрудников на однной должности, в данном подразделении, для соответ. месяца


In [8]:

#Для этого сначала объединим таблицы и добавим доп. столбик со значениями процента от зарплаты полученной на данном сервисе
data = data_work_time.merge(data_service, on = ['subdivision_work', 'position'])
data['money_in_hour'] = (data.persent_use_in_work_time / 100) * data.salary

#После этогосгруппируем таблицу и изменим столбик со значениями процента зарплаты на данном сервисе и разделим на кол-во часов
#Получим столбик со значениями равными зарплате в час полученными на данном сервисе

data_money_in_hour = data.drop(['date', 'employee'], axis = 1) \
    .groupby(['subdivision_work', 'position', 'month', 'service', 'persent_use_in_work_time']) \
    .agg({'working_hours' : 'sum', 'salary' : 'sum', 'money_in_hour' : 'sum'})

#Так как кол-во часов работы может равняться 0, то для заполнения таблицы рассмотрим оба случая
data_money_in_hour = data_money_in_hour.reset_index()
for i in range (data_money_in_hour.shape[0]):
    if data_money_in_hour.working_hours[i] == 0:
        data_money_in_hour.money_in_hour[i] = 0
    if data_money_in_hour.working_hours[i] != 0:
        data_money_in_hour.money_in_hour[i] = round((data_money_in_hour.money_in_hour[i] / data_money_in_hour.working_hours[i]), 2)

<ipython-input-8-22d6e022b012>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_money_in_hour.money_in_hour[i] = round((data_money_in_hour.money_in_hour[i] / data_money_in_hour.working_hours[i]), 2)
<ipython-input-8-22d6e022b012>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_money_in_hour.money_in_hour[i] = 0


#  На основе полученной таблице составим матрицы для определения значимости сервисов

### Создадим матрицу зависимости сервиса от времени работа в различных месяцах

In [11]:
#Создадим матрицу зависимости сервиса от времени работа в различных месяцах

#Создадим доп. таблицу для рассмотрения этих данных, чтобы исходная таблица не подверглась изменениям
data_hours_in_server = data_money_in_hour.copy()
data_hours_in_server['hours_service'] = round((data_hours_in_server.working_hours * data_hours_in_server.persent_use_in_work_time / 100), 2)
data_hours_in_server = data_hours_in_server.groupby(['month', 'service']) \
    .agg({'working_hours' : 'sum'})
data_hours_in_server = data_hours_in_server.reset_index()

#После полученной таблицы сформируем матрицу
matrix = np.array([[0 for i in range(8)] for i in range(8)])

for i in range(7):
    matrix[i+1][0] = list(data_services.keys())[i]
for i in range(7):
    matrix[0][i+1] = i + 1

for i in range(1,8):
    for j in range(1,8):
        matrix[i][j] = data_hours_in_server.working_hours[i * j - 1]

#Полученную матрицу преобразуем в Дата Фрейм который сохраним в отдельный ексель файл
df = pd.DataFrame(matrix, columns = [0, 'January', 'February', 'March', 'April', 'May', 'June', 'July'], index = [0] + list(data_services.values()))
df = df.drop(0, axis = 0)
df = df.drop(0, axis = 1)

df.to_excel('service_month.xlsx')

### Создадим матрицу зависимости часов работы каждого сервиса на соответствующем подразделении
Алгоритм создание таблицы состоит в последовательном заполнении учеек из группированой таблицы data_service_sub. Итоговую таблицу также можно найти в xlsx формете в Приложени - Обработанные данные

In [12]:
#Создадим матрицу зависимости часов работы каждого сервиса на соответствующем подразделении

#Создадим доп. таблицу для рассмотрения этих данных, чтобы исходная таблица не подверглась изменениям
data_money_in_hour = data_money_in_hour.copy()
data_money_in_hour['hours_service'] = round((data_money_in_hour.working_hours * data_money_in_hour.persent_use_in_work_time / 100), 2)
data_service_sub = data_money_in_hour.groupby(['subdivision_work', 'service']) \
                    .agg({'hours_service' : 'sum', 'money_in_hour':'sum'})
data_service_sub = data_service_sub.reset_index()
data_service_sub['money_service']=data_service_sub['hours_service']*data_service_sub['money_in_hour']

#На основе полученных данных составим матрицу
matrix = [[0 for i in range(11)] for i in range(8)]
for i in range(7):
    matrix[i+1][0]=list(data_services.values())[i]
for i in range(10):
    matrix[0][i+1]=list(sub.values())[i]
for i in range(1,8):
    for j in range(1,11):
        matrix[i][j]=data_service_sub.iloc[i*j-1]['hours_service']
        
#Преобразуем матрицу в Дата Фрейм который сохраним в отдельный ексель файл
df = pd.DataFrame(matrix, columns = [0] + list(sub.values()), index = [0] + list(data_services.values()))
df = df.drop(0, axis = 0)
df = df.drop(0, axis = 1)

df.to_csv('cross.csv')

### На основе выше полученных данных составим матрицу зависимости кол-ва средств принесенных каждым сервисом для соответствующих подразделений

Алгоритм создание таблицы аналогичный. Итоговую таблицу также можно найти в xlsx формете в Приложени - Обработанные данные

In [13]:
matrix_money = [[0 for i in range(11)] for i in range(8)]
for i in range(7):
    matrix_money[i+1][0]=list(data_services.values())[i]
for i in range(10):
    matrix_money[0][i+1]=list(sub.values())[i]
for i in range(1,8):
    for j in range(1,11):
        matrix_money[i][j]=data_service_sub.iloc[i*j-1]['money_service']

#Преобразуем матрицу в Дата Фрейм который сохраним в отдельный ексель файл
data_matrix_money = pd.DataFrame(matrix_money, columns = [0] + list(sub.values()), index = [0] + list(data_services.values()))
data_matrix_money = data_matrix_money.drop(0, axis = 0)
data_matrix_money = data_matrix_money.drop(0, axis = 1)

data_matrix_money.to_csv('cross_money.csv')

# Сгруппируем исходные данные, для создания единой плоской таблицы
### Для этого перейдем к исходным данным

In [14]:
data_work_time = pd.read_excel('data.xlsx', sheet_name='Рабочее время')
data_employee = pd.read_excel('data.xlsx', sheet_name='Сотрудник Подр. Месяц')
data_service = pd.read_excel('data.xlsx', sheet_name='Сервисы использование')
data_hierarchy = pd.read_excel('data.xlsx', sheet_name='Иерархия подразделений')

data_work_time = data_work_time.rename(columns = {'Дата' : 'date',
                                'Сотрудник' : 'employee',
                                'Рабочие часы' : 'working_hours',
                                'Подразделение' : 'subdivision_work',
                                'Подразделение_TXT' : 'subdivision_txt'})

data_employee = data_employee.rename(columns = {'Сотрудник' : 'employee',
                                'Подразделение' : 'subdivision_salary',
                                'Подразделение_TXT' : 'subdivision_salary_txt',
                                'Месяц' : 'month',
                                'Должность' : 'position',
                                'Должность_TXT' : 'position_txt',
                                'Пол' : 'gender',
                                'ФИО' : 'full_name',
                                'ЗП (руб.)' : 'salary'})

data_service = data_service.rename(columns = {'Подразделение' : 'subdivision_work',
                               'Подразделение_TXT' : 'subdivision_txt',
                               'Должность' : 'position',
                               'Должность_TXT' : 'position_txt',
                               'Сервис' : 'service',
                               'Сервис_TXT' : 'service_txt',
                               '% использования в рабочее время' : 'persent_use_in_work_time'})

data_hierarchy = data_hierarchy.rename(columns = {'Ид. узла' : 'id_node',
                                 'Ид. подразделения' : 'subdivision_work',
                                 'Наименование' : 'subdivision_txt',
                                 'Родительский узел' : "parent's_node",
                                 'Следущий узел' : 'next_node'})
data_work_time['month'] = [data_work_time['date'].iloc[i].month for i in range(data_work_time.shape[0])]


data_employee['month'] = [data_employee['month'].iloc[i].month for i in range(data_employee.shape[0])]

## Последовательно объединим все таблицы

Также отсортируем их по подразделениям, таким образом можно делать быстрый поиск нужных записей используя алгоритмы с более низкой ассимптотикой. Итоговая таблица есть в xlsx формате в Приложении

In [15]:
full_data = data_work_time.merge(data_employee, on = (['employee', 'month']))
full_data = full_data.merge(data_service, on = (['subdivision_work', 'subdivision_txt', 'position', 'position_txt']))
full_data = full_data.merge(data_hierarchy, on = (['subdivision_work', 'subdivision_txt'])) \
    .sort_values('subdivision_work')

In [18]:
full_data = full_data.drop(columns=['subdivision_salary_txt', 'subdivision_salary'])

In [19]:
full_data.to_excel('Плоская таблица.xlsx')